## Scenario 3: Multiple data scientists workling on multiple ML models

- Tracking server: yes, remote server (EC2)
- Backend store: postgresql database
- Artifacts store: s3 bucket

To run this example you need to launch the mlflow server on your AWS EC2 instance:

`mlflow server -h 0.0.0.0 -p 5000 --backend-store-uri postgresql://<username>:<password>@mlflow-backend-db.c0cccftlr7z4.us-east-1.rds.amazonaws.com:5432/mlflow_db --default-artifact-root s3://agifford-mlflow-artifacts-remote`

The example uses an s3 bucket to store the artifacts, so you'll to provide the bucket. Check the [mlflow documentation](https://mlflow.org/docs/latest/tracking.html#artifact-stores) for more options to store your artifacs.

In [1]:
import mlflow
import os

# os.environ["AWS_PROFILE"] = "" # fill in with your AWS profile. More info: https://docs.aws.amazon.com/sdk-for-java/latest/developer-guide/setup.html#setup-credentials
# i created environment variables using set ... in PowerShell, hopefully this works
TRACKING_SERVER_HOST = "ec2-3-88-231-127.compute-1.amazonaws.com" # CHANGES EVERY TIME YOU RESTART THE EC2 INSTANCE

mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:5000")

In [2]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://ec2-3-88-231-127.compute-1.amazonaws.com:5000'


In [3]:
mlflow.list_experiments()

[<Experiment: artifact_location='s3://agifford-mlflow-artifacts-remote/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>,
 <Experiment: artifact_location='s3://agifford-mlflow-artifacts-remote/1', experiment_id='1', lifecycle_stage='active', name='my-experiment-1', tags={}>]

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

c:\Users\adamgifford_behavr.DESKTOP-PQU0D8M\anaconda3\envs\mlopszoom\lib\site-packages\_distutils_hack\__init__.py:30: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


default artifacts URI: 's3://agifford-mlflow-artifacts-remote/1/fe26da65009c489491125673284d7f97/artifacts'


In [5]:
mlflow.list_experiments()

[<Experiment: artifact_location='s3://agifford-mlflow-artifacts-remote/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>,
 <Experiment: artifact_location='s3://agifford-mlflow-artifacts-remote/1', experiment_id='1', lifecycle_stage='active', name='my-experiment-1', tags={}>]

### Interacting with the model registry

In [6]:
from mlflow.tracking import MlflowClient


client = MlflowClient(f"http://{TRACKING_SERVER_HOST}:5000")

In [7]:
client.list_registered_models()

[]

In [8]:
run_id = client.list_run_infos(experiment_id='1')[0].run_id
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='iris-classifier'
)

Successfully registered model 'iris-classifier'.
2022/06/01 16:17:36 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: iris-classifier, version 1
Created version '1' of model 'iris-classifier'.


<ModelVersion: creation_timestamp=1654114656763, current_stage='None', description='', last_updated_timestamp=1654114656763, name='iris-classifier', run_id='fe26da65009c489491125673284d7f97', run_link='', source='s3://agifford-mlflow-artifacts-remote/1/fe26da65009c489491125673284d7f97/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='1'>